# Init Data

In [56]:
%pylab --no-import-all inline

import gurobipy as grb
import EKPSolvers as ekp

Populating the interactive namespace from numpy and matplotlib


In [84]:
n = 1000
# np.random.seed(20)

## Create Data

In [85]:
def sort_kp(c,w):
    t = [ (c[i],w[i],c[i]/w[i]) for i in range(len(c)) ]
    t.sort(key=lambda x : x[2])
    c = [p[0] for p in t]
    w = [p[1] for p in t]
    return (c,w)

c = 1 + 0.3*np.random.randn(n)
w = np.random.randint(5,12,size=n)+1
x = np.random.randint(2,size=n)
rhs = sum(w*x)

#c,w = sort_kp(c,w)
#c = np.array(c)
#w = np.array(w)

#print(c.tolist(),w.tolist(),x.tolist(),rhs,sep='\n')

# EKP Solver

In [90]:
sol = ekp.SolveEKP_bb(c.tolist(),w.tolist(),rhs)

In [60]:
x_bb = np.array(sol[2])

# Gurobi

In [61]:
c_dict = { i : c[i] for i in range(0,len(c)) }
w_dict = { i : w[i] for i in range(0,len(w)) }

In [62]:
model = grb.Model("knapsack")
vtype = grb.GRB.BINARY
vars = model.addVars(range(0,len(c_dict)),lb=0.0,ub=1.0, \
                     obj=c_dict,vtype=vtype)
model.addConstr(vars.prod(w_dict),grb.GRB.EQUAL,rhs)
model.update()

In [63]:
model.optimize()

Optimize a model with 1 rows, 20 columns and 20 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [6e+00, 1e+01]
  Objective range  [6e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 7e+01]
Presolve time: 0.00s
Presolved: 1 rows, 20 columns, 20 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 7.1682017

Root relaxation: objective 5.312561e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.31256    0    1    7.16820    5.31256  25.9%     -    0s
H    0     0                       5.5557666    5.31256  4.38%     -    0s
     0     0    5.43292    0    2    5.55577    5.43292  2.21%     -    0s
H    0     0                       5.5043761    5.43292  1.30%     -    0s
     0     0    5.47884    0    4    5.50438  

In [64]:
x_gurobi = np.array([v.X for v in vars.values()])

# Enumeration Solver

In [ ]:
x_enum = np.array(ekp.SolveEKP_enum(c,w,rhs))

# Compare Results

In [66]:
print(sol[0],'  <=  ',sol[1])
print('----')

print('{:.5e}  {:.5e}  {:.5e}'.format(np.linalg.norm(x_gurobi - np.round(x_gurobi),ord=1),
                                      np.linalg.norm(x_bb - np.round(x_bb)),
      np.linalg.norm(x_enum - np.round(x_enum))))
print('{:.5e}  {:.5e}  {:.5e}'.format(abs(sum(x_gurobi*w)-rhs),abs(sum(x_bb*w)-rhs),abs(sum(x_enum*w)-rhs)))
print('g: {:.5f}\nb: {:.5f}\ne: {:.5f}'.format(sum(x_gurobi*c),sum(x_bb*c),sum(x_enum*c)))

print('---')
#print(x_bb.astype(int))
#print(x_gurobi.astype(int))
#print(x_enum)

5.5046109768906515   <=   5.504376068977589
----
0.00000e+00  0.00000e+00  0.00000e+00
0.00000e+00  0.00000e+00  0.00000e+00
g: 5.50438
b: 5.50438
e: 5.50438
---
